In [ ]:
# 2020년의 주가를 예측해보자! (by 18/19 데이터)

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

def create_analysis_link(link):
    new = 'https://finance.naver.com' + link
    return 'https://finance.naver.com' + link[:6] + 'coinfo' + link[10:]


links = [] # 최종적인 링크들을 담을 테이블
# 시가총액 1~200위 페이지
for page in range(1,37):
    html = urlopen('https://finance.naver.com/sise/sise_market_sum.naver?&page='+str(page))
    bs = BeautifulSoup(html, 'html.parser')
    rankings = bs.findAll('a',{'href' : re.compile('^(/item/main.)')})

    # 각 종목별 페이지로 연결
    for ranking in rankings:
            if 'href' in ranking.attrs:
                midLink = str(create_analysis_link(ranking.attrs['href']))
                
                # 종목별 페이지 속의 '재무분석' 페이지로 연결
                html = urlopen("{}".format(midLink))
                bs = BeautifulSoup(html,'html.parser')
                link = bs.find('div', {'class' : 'section inner_sub'}).find('iframe')
                
                if 'src' in link.attrs:
                    financialLink = link.attrs['src']
                    links.append(financialLink)
                    


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import time

data = []
stocks = []

for one in links:
    # headless 설정
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    # 브라우저 생성
    browser = webdriver.Chrome(chrome_options=options)
    browser.get('{}'.format(one))
    
    try: # ETF를 걸러줄 장치
    
        # 종목 이름 구하기
        elementXpath = '//*[@id="pArea"]/div[1]/div/table/tbody/tr[1]/td/dl/dt[1]/span'
        title = browser.find_element(by=By.XPATH, value = elementXpath).text

        # 포괄손익계산서 이동하기
        browser.find_element_by_link_text("재무분석").click()
        browser.find_element_by_link_text("포괄손익계산서").click()

        time.sleep(1)
        # 영업이익 비교하기_ 조건 불합시 continue
        try:
            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[58]/td[4]'
            O_profit2019 = browser.find_element(by=By.XPATH, value = elementXpath).text

            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[58]/td[3]'
            O_profit2018 = browser.find_element(by=By.XPATH, value = elementXpath).text

            if O_profit2019 <= O_profit2018:
                browser.quit()
                continue

        except: # 영업이익이 안뜬다면.
            browser.quit()
            continue

        # 재무상태표 이동하기
        browser.find_element_by_link_text("재무상태표").click()
        time.sleep(1)

        # 순운전자본 구하기
        elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[2]/td[4]'
        current_asset = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

        elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[128]/td[4]'
        current_liabilities = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

        elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[47]/td[4]'
        cashable_asset = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

        net_working_capital = current_asset - current_liabilities + cashable_asset

        # 조건으로부터 살아남았다면, 수익률 계산(추가 작성 예정)

        # 조건으로부터 살아남았다면, 리스트 테이블에 추가
        data.append(net_working_capital) 
        stocks.append(title)
        
    except: # ETF는 거른다.
        continue


# 브라우저 생성
options = webdriver.ChromeOptions()
options.add_argument('headless')
# chrome_options=options
browser = webdriver.Chrome()
browser.get('{}'.format('http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101'))

# 개별 종목 시세 추이 로 이동
elementXpath = '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[2]/ul/li[1]/ul/li[3]/a'
WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
time.sleep(1)
browser.find_element(by = By.XPATH, value = elementXpath).click()

# 검색 옵션 선택
time.sleep(3)

content = []
i = 0
for stock in stocks:
    # 검색어 입력
    search_box = browser.find_element(by = By.XPATH , value = "//*[@id='tboxisuCd_finder_stkisu0_1']")
    browser.find_element(by = By.XPATH , value = "//*[@id='tboxisuCd_finder_stkisu0_1']").clear()
    search_box.send_keys("{}".format(stock))

        # 새로운 창에서의 클릭
    search_box.send_keys(Keys.RETURN)
    browser.switch_to.window(browser.window_handles[-1])

    time.sleep(3)
    browser.find_element(by = By.XPATH , value = "//*[@id='jsGrid__finder_stkisu0_1']/tbody/tr[1]/td[2]").click()

        # 다시 원래 창으로 돌아옴.
    browser.switch_to.window(browser.window_handles[0])
    time.sleep(3)

    # 날짜 선택 먼저 20191231
 
    box = browser.find_element(by = By.XPATH , value = " //*[@id='endDd']")
    box.clear()
    box.send_keys("20191231")
    
    browser.find_element(by = By.XPATH , value = "//*[@id='MDCSTAT017_FORM']/div[1]/div/table/tbody/tr[2]/td/div/div/button[2]").click()
    
    # '조회' 버튼 클릭
        # 여기서 다양하게 시도를 해서 시간이 많이 걸렸는데, 이상하게 두번째버튼만 클릭하면 에러가 난다.
        # 인터넷을 참고해서 아무 버튼이나 클릭한 뒤에 클릭하도록 했더니 실제로 된다...
    browser.find_element(by = By.XPATH , value = "//*[@id='jsMdiTab']/li[1]/a/button").click()
    browser.find_element(by = By.XPATH , value = "//*[@id='jsSearchButton']").click()

    # 출력
    elementXpath = "//*[@id='jsMdiContent']/div/div[1]/div[1]/div[1]/div[2]/div/div/table/tbody/tr/td[10]"
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
    time.sleep(1)

    cap = float(browser.find_element(by = By.XPATH , value = elementXpath).text.replace(",",""))

    if cap >= data[i]:
        continue
    rate = round(cap/data[i]*100,2)
    
    stock_after = float(browser.find_element(by = By.XPATH , value = "//*[@id='jsMdiContent']/div/div[1]/div[1]/div[1]/div[2]/div/div/table/tbody/tr/td[2]").text.replace(",",""))

    # 날짜 선택 먼저 20191231
   
    box = browser.find_element(by = By.XPATH , value = " //*[@id='endDd']")
    box.clear()
    box.send_keys("20181231")
    
    browser.find_element(by = By.XPATH , value = '//*[@id="MDCSTAT017_FORM"]/div[1]/div/table/tbody/tr[2]/td/div/div/button[2]').click()
    
    # '조회' 버튼 클릭
        # 여기서 다양하게 시도를 해서 시간이 많이 걸렸는데, 이상하게 두번째버튼만 클릭하면 에러가 난다.
        # 인터넷을 참고해서 아무 버튼이나 클릭한 뒤에 클릭하도록 했더니 실제로 된다...
    browser.find_element(by = By.XPATH , value = "//*[@id='jsMdiTab']/li/a/span").click()
    browser.find_element(by = By.XPATH , value = "//*[@id='jsSearchButton']").click()

    # 출력
    elementXpath = '//*[@id="jsMdiContent"]/div/div[1]/div[1]/div[1]/div[2]/div/div/table/tbody/tr/td[2]'
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
    time.sleep(1)

    stock_before = float(browser.find_element(by = By.XPATH , value = elementXpath).text.replace(",",""))
    earning_rate = round((stock_after - stock_before)/stock_before*100,2)
    content.append((stock, rate, stock_before, stock_after, earning_rate))
    i += 1
    
# 데이터 저장
import csv

csvFile = open('Cospi2019 분석 자료.csv', 'wt+', newline = '')
writer = csv.writer(csvFile)

writer.writerow(('기업 이름', '시가총액/순운전자본', '2018/12 주가', '2019/12 주가', '수익률'))

try:
    for one in data:
        writer.writerow(one)
        
finally:
    csvFile.close()
